This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

# Parametric Polynomial Curves

The building blocks for
*polynomial splines*
are of course [polynomials](https://en.wikipedia.org/wiki/Polynomial).

But first things first,
let's import [SymPy](https://www.sympy.org/)
and a few helper functions from [helper.py](helper.py):

In [ ]:
import sympy as sp
sp.init_printing(order='grevlex')
from helper import plot_basis, plot_sympy, grid_lines, plot_spline_2d

We are mostly interested in *univariate* splines,
i.e. curves with one free parameter,
which are built using polynomials with a single parameter.
Here we are calling this parameter $t$.
You can think about it as *time* (e.g. in seconds),
but it doesn't have to represent time.

In [ ]:
t = sp.symbols('t')

Polynomials typically consist of multiple *terms*.
Each term contains a *basis function*,
which itself contains one or more integer powers of $t$.
The highest power of all terms is called the *degree* of the polynomial.

The arguably simplest set of basis functions
is the *monomial basis*, a.k.a. *power basis*,
which simply consists of all powers of $t$ up to the given degree:

In [ ]:
b_monomial = sp.Matrix([t**3, t**2, t, 1]).T
b_monomial

In this example we are creating polynomials of degree 3,
which are also called *cubic* polynomials.

The ordering of the basis functions is purely a matter of convention,
here we are sorting them in order of descending powers.

These basis functions are multiplied by (constant) *coefficients*.
We are writing the coefficients with bold symbols,
because apart from simple scalars (for one-dimensional functions),
these symbols can also represent vectors in two- or three-dimensional space
(and even higher-dimensional spaces).

In [ ]:
coefficients = sp.Matrix(sp.symbols('a:dbm')[::-1])
coefficients

We can create a polynomial by
multiplying the basis functions with the coefficients
and then adding all terms:

In [ ]:
b_monomial.dot(coefficients)

This is a cubic polynomial in its *canonical form*
(because it uses monomial basis functions).

Let's take a closer look at those basis functions:

In [ ]:
plot_basis(*b_monomial)

It doesn't look like much,
but every conceivable cubic polynomial
can be expressed as exactly one linear combination
of those basis functions
(i.e. using one specific list of coefficients).

An example polynomial that's not in canonical form ...

In [ ]:
example_polynomial = (2 * t - 1)**3 + (t + 1)**2 - 6 * t + 1
example_polynomial

In [ ]:
plot_sympy(example_polynomial, (t, 0, 1))
grid_lines([0, 1], [0, 0.5, 1])

... can simply be re-written with monomial basis functions:

In [ ]:
example_polynomial.expand()

Any polynomial can be rewritten using any set of basis functions
(as long as the degree of the basis function set matches the degree of the polynomial).

In later sections we will see more basis functions,
for example those that are used for
[Hermite](hermite-uniform.ipynb),
[Bézier](bezier-de-casteljau.ipynb) and
[Catmull--Rom](catmull-rom-uniform.ipynb) splines.
In those sections we will also see how to convert
between different bases by means of matrix multiplication.

In the previous example,
we used scalar coefficients
to create a one-dimensional polynomial.
We can use two-dimensional coefficients
to create two-dimensional polynomial curves.
Let's create a little class to try this:

In [ ]:
import numpy as np

class CubicPolynomial:
    
    grid = 0, 1
    
    def __init__(self, d, c, b, a):
        self.coeffs = d, c, b, a

    def evaluate(self, t):
        t = np.expand_dims(t, -1)
        return t**[3, 2, 1, 0] @ self.coeffs

<div class="alert alert-info">

Note

The `@` operator is used here to do
[NumPy's matrix multiplication](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html).

</div>

In [ ]:
poly_2d = CubicPolynomial([-1.5, 5], [1.5, -8.5], [1, 4], [3, 2])

Since this class has the same interface as the splines
that will be discussed in later sections,
we can use a spline helper function for plotting:

In [ ]:
plot_spline_2d(poly_2d, dots_per_second=30, chords=False)

This class can also be used with three and more dimensions.
The class [splines.Monomial](../python-module/splines.rst#splines.Monomial)
can be used to try this with arbitrary polynomial degree.